In [22]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader.famafrench import get_available_datasets,FamaFrenchReader
import pandas_datareader.data as web
import warnings
import statsmodels.api as sm


warnings.simplefilter(action="ignore", category=FutureWarning)

tickers = [
    'DIS', 'UHS', 'GOOGL', 'LOW', 'BSX', 'JPM', 'EOG', 'SBAC', 'STT', 'DVN',
    'VRTX', 'BK', 'ALL', 'WY', 'OXY', 'ON', 'EXC', 'UAL', 'AKAM', 'SMCI',
    'LYV', 'AMD', 'VLO', 'NI', 'DRI', 'SJM', 'APH', 'MAS', 'CMG', 'TXT'
]

n_tickers = len(tickers)

start_date = '2014-01-01'
end_date = '2023-12-31'

# Download the daily data from stocks
price = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
daily_returns_stocks = price.pct_change()
daily_returns_stocks = daily_returns_stocks.dropna()

# Download factors. All data is monthly, so it is necessary to change the returns to be monthly returns
research_factors = FamaFrenchReader("F-F_Research_Data_5_Factors_2x3_daily",start=pd.to_datetime(start_date),end=pd.to_datetime(end_date)).read()[0]
industry_factors = FamaFrenchReader("5_Industry_Portfolios_daily",start=pd.to_datetime(start_date),end=pd.to_datetime(end_date)).read()[0] # Average Value Weighted Returns -- Monthly
momentum_factor = FamaFrenchReader("F-F_Momentum_Factor_daily",start=pd.to_datetime(start_date),end=pd.to_datetime(end_date)).read()[0]

# SMB -> Factor for Size
# HML -> Factor for B/M
# RNW -> Factor for profitabilty
# CMA -> Factor for growth
# Mom -> Factor for Momentum
# Mkt -> Factor for Mkt

# Merge all factors
factors = pd.concat([research_factors,industry_factors,momentum_factor],axis=1)
factors["Mkt"] = factors["Mkt-RF"] + factors["RF"]
factors = factors.drop(columns=["Mkt-RF","RF"])

factors = factors.loc[:].div(100)
factors = factors.iloc[1:,:]

In [24]:
def fit_factor_model(returns, factors):
    result = sm.OLS(returns, factors).fit()
    return result.params

betas = pd.DataFrame(index=daily_returns_stocks.columns, columns = factors.columns)
for stock in daily_returns_stocks.columns:
    betas.loc[stock] = fit_factor_model(daily_returns_stocks[stock], factors)
    
betas

,SMB,HML,RMW,CMA,Cnsmr,Manuf,HiTec,Hlth,Other,Mom,Mkt
Ticker,,,,,,,,,,,
AKAM,-0.083095,-0.344806,0.020868,0.49915,0.648017,0.377111,1.348498,0.512201,0.712868,-0.023157,-2.682119
ALL,-0.196746,0.154539,0.118711,0.04827,0.466787,0.397941,0.305872,0.30302,1.014625,0.098887,-1.591492
AMD,0.388157,-0.350773,-0.172973,0.177984,-1.15372,-0.737453,-0.262645,-0.872384,-1.306198,-0.024335,5.627825
APH,0.197369,-0.355164,0.286222,0.081508,0.022454,0.50966,0.620262,0.002767,0.663712,0.004531,-0.841532
BK,-0.163061,0.51054,-0.268196,-0.050963,0.113806,-0.29472,0.138738,0.153063,1.225552,-0.068121,-0.367374
BSX,0.035865,-0.256013,0.218102,-0.068803,0.645684,0.959941,1.499104,1.050573,1.404014,0.033969,-4.518115
CMG,0.22178,-0.459424,0.008916,-0.092196,-0.259999,-0.776074,-1.248511,-0.701388,-0.320847,0.068263,4.08668
DIS,-0.026354,-0.197847,-0.167979,0.04029,0.89631,0.704444,1.566111,0.160973,1.58841,-0.20609,-4.034268
DRI,0.47486,-0.54939,0.569,-0.06086,0.567441,0.308859,0.147954,-0.17093,1.797153,0.005697,-1.635998


In [25]:
factor_cov_matrix = factors.cov()
factor_cov_matrix

,SMB,HML,RMW,CMA,Cnsmr,Manuf,HiTec,Hlth,Other,Mom,Mkt
SMB,4.344018e-05,0.000014,-0.000009,5.838998e-08,0.000011,2.107972e-05,0.000008,0.000011,0.000025,-0.000021,0.000015
HML,1.377481e-05,0.000077,0.000015,2.469025e-05,-0.000015,2.804046e-05,-0.000032,-0.000019,0.000028,-0.000030,-0.000006
RMW,-8.804880e-06,0.000015,0.000026,7.435144e-06,-0.000008,-4.482926e-06,-0.000014,-0.000015,-0.000007,-0.000002,-0.000011
CMA,5.838998e-08,0.000025,0.000007,2.027478e-05,-0.000017,-2.690663e-09,-0.000027,-0.000014,-0.000009,0.000002,-0.000016
Cnsmr,1.095602e-05,-0.000015,-0.000008,-1.702499e-05,0.000119,1.014069e-04,0.000130,0.000091,0.000116,-0.000013,0.000117
Manuf,2.107972e-05,0.000028,-0.000004,-2.690663e-09,0.000101,1.414455e-04,0.000118,0.000094,0.000137,-0.000033,0.000121
HiTec,8.086787e-06,-0.000032,-0.000014,-2.707016e-05,0.000130,1.183661e-04,0.000179,0.000114,0.000138,-0.000010,0.000146
Hlth,1.115715e-05,-0.000019,-0.000015,-1.363030e-05,0.000091,9.355827e-05,0.000114,0.000124,0.000107,-0.000005,0.000106
Other,2.518210e-05,0.000028,-0.000007,-8.861398e-06,0.000116,1.372619e-04,0.000138,0.000107,0.000167,-0.000036,0.000137
Mom,-2.121751e-05,-0.000030,-0.000002,2.067999e-06,-0.000013,-3.343592e-05,-0.000010,-0.000005,-0.000036,0.000118,-0.000019


In [26]:
specific_variances = pd.Series(index=daily_returns_stocks.columns)

for stock in daily_returns_stocks.columns:
    residuals = daily_returns_stocks[stock] - factors.dot(betas.loc[stock])
    specific_variances[stock] = residuals.var()
    
estimate_with_factors_cov_matrix = betas.dot(factor_cov_matrix).dot(betas.T) + pd.DataFrame(np.diag(specific_variances), index=daily_returns_stocks.columns, columns=daily_returns_stocks.columns)

In [27]:
estimate_with_factors_cov_matrix

Ticker,AKAM,ALL,AMD,APH,BK,BSX,CMG,DIS,DRI,DVN,...,SBAC,SJM,SMCI,STT,TXT,UAL,UHS,VLO,VRTX,WY
Ticker,,,,,,,,,,,,,,,,,,,,,
AKAM,0.00039,0.000076,0.000194,0.000114,0.000093,0.000109,0.000103,0.000103,0.000106,0.000115,...,0.000095,0.000045,0.000137,0.000117,0.00012,0.000125,0.000104,0.000097,0.000128,0.000122
ALL,0.000076,0.000221,0.000118,0.000109,0.00014,0.000105,0.000074,0.000109,0.000136,0.000188,...,0.000076,0.00005,0.000121,0.000166,0.000159,0.00018,0.000132,0.000159,0.000089,0.000143
AMD,0.000194,0.000118,0.001342,0.000218,0.000164,0.000174,0.000219,0.000199,0.000199,0.000256,...,0.00016,0.000041,0.000294,0.000216,0.000224,0.000247,0.000168,0.000176,0.000189,0.000223
APH,0.000114,0.000109,0.000218,0.000216,0.000141,0.000129,0.00012,0.000139,0.000159,0.000212,...,0.000103,0.000046,0.000179,0.000177,0.000184,0.000204,0.000146,0.000167,0.000119,0.000172
BK,0.000093,0.00014,0.000164,0.000141,0.000291,0.000131,0.000102,0.000153,0.000186,0.000255,...,0.000087,0.000043,0.00017,0.00024,0.000214,0.000269,0.000173,0.000209,0.000107,0.000185
BSX,0.000109,0.000105,0.000174,0.000129,0.000131,0.000293,0.000096,0.000119,0.000137,0.00017,...,0.000101,0.000053,0.000151,0.000159,0.000161,0.000171,0.000148,0.000152,0.000151,0.000152
CMG,0.000103,0.000074,0.000219,0.00012,0.000102,0.000096,0.000477,0.000114,0.000129,0.000117,...,0.000086,0.000026,0.000158,0.000131,0.000127,0.000159,0.000103,0.00009,0.000096,0.00013
DIS,0.000103,0.000109,0.000199,0.000139,0.000153,0.000119,0.000114,0.000293,0.000162,0.000212,...,0.000094,0.000039,0.000167,0.000188,0.000183,0.000223,0.000143,0.000167,0.0001,0.000168
DRI,0.000106,0.000136,0.000199,0.000159,0.000186,0.000137,0.000129,0.000162,0.000492,0.000236,...,0.000095,0.000048,0.000192,0.000228,0.000225,0.000286,0.000184,0.0002,0.0001,0.000204


In [28]:
regular_matrix_cov = daily_returns_stocks.cov()
regular_matrix_cov

Ticker,AKAM,ALL,AMD,APH,BK,BSX,CMG,DIS,DRI,DVN,...,SBAC,SJM,SMCI,STT,TXT,UAL,UHS,VLO,VRTX,WY
Ticker,,,,,,,,,,,,,,,,,,,,,
AKAM,0.000390,0.000077,0.000189,0.000118,0.000101,0.000102,0.000096,0.000100,0.000071,0.000125,...,0.000114,0.000062,0.000131,0.000116,0.000121,0.000106,0.000089,0.000090,0.000142,0.000108
ALL,0.000077,0.000221,0.000107,0.000111,0.000138,0.000107,0.000080,0.000105,0.000141,0.000175,...,0.000082,0.000058,0.000108,0.000163,0.000163,0.000174,0.000135,0.000156,0.000079,0.000141
AMD,0.000189,0.000107,0.001340,0.000222,0.000172,0.000173,0.000223,0.000179,0.000179,0.000279,...,0.000153,0.000040,0.000318,0.000233,0.000216,0.000227,0.000154,0.000190,0.000202,0.000219
APH,0.000118,0.000111,0.000222,0.000216,0.000144,0.000132,0.000122,0.000130,0.000159,0.000200,...,0.000104,0.000044,0.000184,0.000180,0.000193,0.000214,0.000147,0.000162,0.000108,0.000172
BK,0.000101,0.000138,0.000172,0.000144,0.000291,0.000128,0.000094,0.000151,0.000175,0.000258,...,0.000081,0.000045,0.000167,0.000299,0.000214,0.000248,0.000167,0.000208,0.000109,0.000176
BSX,0.000102,0.000107,0.000173,0.000132,0.000128,0.000292,0.000111,0.000123,0.000159,0.000168,...,0.000103,0.000053,0.000153,0.000156,0.000162,0.000193,0.000179,0.000164,0.000141,0.000156
CMG,0.000096,0.000080,0.000223,0.000122,0.000094,0.000111,0.000477,0.000119,0.000194,0.000160,...,0.000088,0.000009,0.000116,0.000120,0.000106,0.000178,0.000124,0.000096,0.000099,0.000129
DIS,0.000100,0.000105,0.000179,0.000130,0.000151,0.000123,0.000119,0.000294,0.000190,0.000225,...,0.000090,0.000034,0.000160,0.000186,0.000191,0.000251,0.000149,0.000171,0.000100,0.000167
DRI,0.000071,0.000141,0.000179,0.000159,0.000175,0.000159,0.000194,0.000190,0.000491,0.000275,...,0.000089,0.000034,0.000173,0.000214,0.000231,0.000358,0.000237,0.000240,0.000098,0.000221
